In [27]:
#Write python code that prints total sum of money in this list

amounts = [
    "   1,000.31   doLLaRs   ",
    "500 Dollars  ",
    "25.03 USD",
    "$10"
]


#Need to assign a variable to new list

for amount in amounts:
    clean = amount.strip()
    clean = clean.replace(",", "")
    print(clean)
    

#Could also choose to chain the methods instead of reassigning to same varaiblae
print ("---")

amount_floats = []
for amount in amounts:
    clean = (
        amount
        .strip(" $")
        .replace(",", "")
        .split(" ")[0]
    )
    amount_floats.append(float(clean))
    
sum(amount_floats)


1000.31   doLLaRs
500 Dollars
25.03 USD
$10
---


1535.34

In [28]:
#alternative solution
# '+=' notation: means add to current value

total = 0
for amount in amounts:
    clean = (
        amount
        .strip(" $")
        .replace(",", "")
        .split(" ")[0]
    )
    total += float(clean)
    
print(total)

1535.34


## New Exercise, Part 3 - with a twist

In [37]:
#My attempt
amounts = [
    "   1,000.31 doLLaRs   ",
    "54 cents   ",
    "33 CENTS",
    "$10"
]

for amount in amounts:
    clean = (
        amount
        .strip("   $")
        .replace("cents", "")
        .replace("CENTS", "")
        .replace(",", "")
    )
print(clean)

10


In [38]:
#Solution: create your own function
def clean_amount(amt):
    if "$" in amt or "dollar" in amt.lower():
        conversion = 1
    elif "cent" in amt.lower():
        conversion = 0.01
    else:
        raise ValueError(f"Cannot determine unit for {amt}")
        
    amt = amt.strip("$ ")
    amt = amt.replace(",", "")
    amt = amt.split(" ")[0]
    return float(amt) * conversion

sum(clean_amount(a) for a in amounts)

1011.18

In [45]:
#Simpler version of above to understand how to create own function
#For example, would use this to clean all data

def clean_amt(amt):
    amt = amt.strip("$  ")
    amt = amt.replace(",", "")
    amt = amt.split("  ")[0]
    return float(amt)

clean_amt("1,300.50")

1300.5

In [46]:
import pandas as pd

In [49]:
series = pd.Series([ "$1,300.50", "100 dollars", "20 dolLLLLars"])
series

0        $1,300.50
1      100 dollars
2    20 dolLLLLars
dtype: object

In [50]:
#example of cleaning data using function above- refer to Jeremy's notebook
series.apply(clean_amt)

ValueError: could not convert string to float: '100 dollars'

In [51]:

amounts = pd.Series([
    "   1,000.31   doLLaRs   ",
    "54 cents  ",
    "33 CENTS",
    "$10"
])

amounts.str.strip("$ ")

0    1,000.31   doLLaRs
1              54 cents
2              33 CENTS
3                    10
dtype: object

In [52]:
amounts.str.split(" ").str.get(-1)

0         
1         
2    CENTS
3      $10
dtype: object

In [54]:
# Pandas version: use '.get()' to identify specific item in list, equivalent to []
# Combination of all cleaning
# Parentheses on the outside allow you to do chaining
(
    amounts
    .str.strip("$ ")
    .str.replace(",", "")
    .str.split()
    .str.get(0)
)

0    1000.31
1         54
2         33
3         10
dtype: object

In [55]:
def get_conversion(amt):
    if "$" in amt or "dollar" in amt.lower():
        conversion = 1
    elif "cent" in amt.lower():
        conversion = 0.01
    return conversion

#test function
get_conversion("1 dollar")

1

In [56]:
amounts.apply(get_conversion)

0    1.00
1    0.01
2    0.01
3    1.00
dtype: float64

In [57]:
# Convert to dataframe

amounts_df = pd.DataFrame({
    "raw": amounts
})

amounts_df

,raw
0,"1,000.31 doLLaRs"
1,54 cents
2,33 CENTS
3,$10


In [58]:
#Adding cleaned data


    .str.split()
    .str.get(0)
    .astype(float)
)

amounts_df

,raw,quantity
0,"1,000.31 doLLaRs",1000.31
1,54 cents,54.00
2,33 CENTS,33.00
3,$10,10.00


In [59]:
amounts_df["conversion"] = amounts_df["raw"].apply(get_conversion)

In [60]:
amounts_df["dollars"] = amounts_df["quantity"] * amounts_df["conversion"]

In [61]:
amounts_df

,raw,quantity,conversion,dollars
0,"1,000.31 doLLaRs",1000.31,1.00,1000.31
1,54 cents,54.00,0.01,0.54
2,33 CENTS,33.00,0.01,0.33
3,$10,10.00,1.00,10.00


In [ ]:
# 'lambda' - unnamed fucntion. whatever the dataframe is, do x (whatever
#the following code is)

In [62]:
#Jeremy's style [advanced]

(
    pd.DataFrame({ "raw": amounts })
    .assign(
        quantity = lambda df: (
            df["raw"]
            .str.strip("$ ")
            .str.replace(",", "")
            .str.split()
            .str.get(0)
            .astype(float)
        ),
        conversion = lambda df: df["raw"].apply(get_conversion),
        dollars = lambda df: df["quantity"] * df["conversion"],
    )
)

,raw,quantity,conversion,dollars
0,"1,000.31 doLLaRs",1000.31,1.00,1000.31
1,54 cents,54.00,0.01,0.54
2,33 CENTS,33.00,0.01,0.33
3,$10,10.00,1.00,10.00
